# LSTM模型构造

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Activation, Dropout 
from tensorflow.keras.callbacks import LearningRateScheduler

model = Sequential([
    LSTM(units=32, input_shape=(32, 1), return_sequences=False),
    Dense(units=16, activation='relu'),
    Dense(units=2, activation='softmax')
])


def lr_schedule(epoch):
    initial_lr = 0.001  # 初始学习率
    decay_factor = 0.5  # 学习率衰减因子
    decay_epochs = 5    # 学习率衰减的周期数
    new_lr = initial_lr * (decay_factor ** (epoch // decay_epochs))
    return new_lr

# 定义学习率回调
lr_scheduler = LearningRateScheduler(lr_schedule)


# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



# 打印模型概况
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 64)            57600     
                                                                 
 lstm (LSTM)                 (None, 32, 128)           98816     
                                                                 
 activation (Activation)     (None, 32, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 205954 (804.51 KB)
Trainable params: 20595

## 导入可直接进行训练的数据

In [7]:
data = pd.read_excel('./test.xlsx') # 需要训练哪个数据集，就把哪个数据集放到test中，这些数据在《可直接训练的数据当中》
data

,id,第1个行为,第2个行为,第3个行为,第4个行为,第5个行为,第6个行为,第7个行为,第8个行为,第9个行为,...,第24个行为,第25个行为,第26个行为,第27个行为,第28个行为,第29个行为,第30个行为,第31个行为,第32个行为,逻辑风格
0,0,11,1,3,8,7,5,3,8,9,...,5,2,8,7,0,0,0,0,0,2
1,1,11,1,2,4,4,4,3,8,5,...,7,5,7,0,0,0,0,0,0,2
2,2,11,1,3,8,11,5,2,3,11,...,8,6,0,0,0,0,0,0,0,2
3,3,11,1,3,8,7,5,3,8,9,...,3,7,5,4,3,0,0,0,0,1
4,4,8,11,1,2,4,4,4,2,3,...,3,9,5,3,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,127,8,11,1,3,8,10,5,2,4,...,2,3,11,5,4,7,0,0,0,2
896,128,8,11,1,3,8,10,5,3,8,...,5,2,3,11,5,3,0,0,0,2
897,129,11,8,1,3,8,11,5,2,7,...,5,3,8,11,5,3,0,0,0,1
898,130,11,8,1,2,4,4,4,2,3,...,5,3,7,8,5,7,0,0,0,2


In [8]:
train_data = data.iloc[0:631,1:34]
val_data = data.iloc[631:768,1:34]
test_data = data.iloc[768:900,1:34]

In [9]:
# 划分数据集
X_train = train_data.iloc[:,0:32]
y_train = train_data.iloc[:,32]-1 # 后面进行one-hot 
X_val = val_data.iloc[:,0:32]
y_val= val_data.iloc[:,32]-1 # 后面进行one-hot 
X_test = test_data.iloc[:,0:32]
y_test = test_data.iloc[:,32]-1 # 后面进行one-hot 

In [10]:
display(X_train.shape,y_train.shape)

(631, 32)

(631,)

In [11]:
display(X_val.shape,y_val.shape)

(137, 32)

(137,)

## 数据训练

In [12]:
# 假设你已经准备好了 train_data 和 train_labels 数据

# 进行标签的 one-hot 编码
from tensorflow.keras.utils import to_categorical
y_train_encoded = to_categorical(y_train, num_classes=2)
y_val_encoded = to_categorical(y_val, num_classes=2)
# 训练模型
batch_size = 12
epochs = 100

history = model.fit(X_train, y_train_encoded, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val_encoded), callbacks=[lr_scheduler])
# history = model.fit(train_data, y_train_encoded, batch_size=batch_size, epochs=epochs, callbacks=[lr_scheduler])


Epoch 1/100
53/53 [==============================] - 9s 51ms/step - loss: 0.5184 - accuracy: 0.7480 - val_loss: 0.5638 - val_accuracy: 0.7080 - lr: 0.0010
Epoch 2/100
53/53 [==============================] - 2s 34ms/step - loss: 0.3159 - accuracy: 0.8605 - val_loss: 0.5566 - val_accuracy: 0.7591 - lr: 0.0010
Epoch 3/100
53/53 [==============================] - 2s 34ms/step - loss: 0.3273 - accuracy: 0.8526 - val_loss: 0.5425 - val_accuracy: 0.7080 - lr: 0.0010
Epoch 4/100
53/53 [==============================] - 2s 35ms/step - loss: 0.2640 - accuracy: 0.8748 - val_loss: 0.5689 - val_accuracy: 0.6934 - lr: 0.0010
Epoch 5/100
53/53 [==============================] - 2s 32ms/step - loss: 0.2233 - accuracy: 0.8986 - val_loss: 0.4968 - val_accuracy: 0.7153 - lr: 0.0010
Epoch 6/100
53/53 [==============================] - 2s 32ms/step - loss: 0.2186 - accuracy: 0.9081 - val_loss: 0.5598 - val_accuracy: 0.7299 - lr: 5.0000e-04
Epoch 7/100
53/53 [==============================] - 2s 32ms/step 